In [6]:
import mechanicalsoup
import requests
import logging

logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

In [11]:
class CommercialRegisterRetriever:
    def __init__(self, session_id: str = None, company_name: str = None):
        retriever.browser.browser.browser.browser = mechanicalsoup.StatefulBrowser(user_agent='Chrome')

        if session_id is None:
            retriever.browser.open("https://www.unternehmensregister.de/ureg/index.html")
            url = retriever.browser.page.select("ul.service__list li a")[0].attrs["href"]
            retriever.session_id = url.split(";")[1].split("?")[0]
       
        retriever.company = {}
        retriever.gs_date = ""
        retriever.gs_file_name = ""

    def search(self, company_name: str):
        # Fill-in the search form
        retriever.browser.select_form('#globalSearchForm')
        retriever.browser["globalSearchForm:extendedResearchCompanyName"] = company_name
        retriever.browser["submitaction"] = "searchRegisterData"
        retriever.browser.submit_selected(btnName="globalSearchForm:btnExecuteSearchOld")

        # retriever.browser.open(
        #     "https://www.unternehmensregister.de/ureg/search1.1.html;{}".format(
        #         retriever.session_id))
        # retriever.browser.select_form("#searchRegisterForm")
        # retriever.browser["searchRegisterForm:extendedResearchCompanyName"] = company_name
        # retriever.browser["searchRegisterForm:extendedResearchRegisterNumber"] = company_id
        # # retriever.browser["searchRegisterForm:extendedResearchStartDate"] = "01.01.2014"

        # retriever.browser["submitaction"] = "searchExtendedResearch"
        # retriever.browser.submit_selected()

        # page = retriever.browser.page
        # # select first company in the list
        # links = retriever.browser.page.find_all('a', href=lambda href: href and 'registerPortalAdvice.html' in href)
        # retriever.browser.open_relative(links[0].attrs["href"])
        

        # open the search results
        retriever.browser.open_relative(retriever.browser.page.select("div.right a")[0].attrs["href"])

        # retrieve all companies in a list
        companies = []

        # get si links
        si = retriever.browser.page.select(".reglink[id*='SI']")

        # get company information
        results_page = retriever.browser.page
        results = results_page.find("tbody").find_all("tr", attrs={"class": None})

        for i in range(0, len(results), 2):
            company = {}
            company["court_city"] = results[i].find("td", attrs={"class": "RegPortErg_AZ"}).text.split("\n")[0]
            company["court"] = results[i].find("td", attrs={"class": "RegPortErg_AZ"}).text.split("\xa0")[2].strip()
            company["id"] = results[i].find("td", attrs={"class": "RegPortErg_AZ"}).text.split("\xa0")[3].strip() + " " + results[i].find("td", attrs={"class": "RegPortErg_AZ"}).text.split("\xa0")[4].strip() 
            company["name"] = results[i+1].find("td", attrs={"class": "RegPortErg_FirmaKopf"}).text.strip()
            company["city"] = results[i+1].find("td", attrs={"class": "RegPortErg_SitzStatusKopf"}).text.strip()
            company["status"] = results[i+1].findAll("td", attrs={"class": "RegPortErg_SitzStatusKopf"})[1].text.strip()
            company["search_index"] = int(i/2)
            company["document_urls"] = {"si": "https://www.unternehmensregister.de/ureg/registerPortal.html;{}{}".format(retriever.session_id, si[int(i/2)].attrs["href"]),}

            companies.append(company)
            i+=1
            
        return companies
        

In [12]:
retriever = CommercialRegisterRetriever()

In [13]:
result = retriever.search("tanso")

In [15]:
result

[{'court_city': 'Bayern',
  'court': 'München',
  'id': 'HRB 269123',
  'name': 'Tanso Technologies GmbH',
  'city': 'München',
  'status': 'Aktuell',
  'search_index': 0,
  'document_urls': {'si': 'https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid=9647EC9778637BD3D82EB7EFFA09F205.web04-1?submitaction=check4duplicate&iddok=null&ausdruckart=SI&gkz=D2601&gkzalt=&rtype=HRB&rnr=269123&cname=Tanso+Technologies+GmbH&state=2&location=&uniqueid=1689541875694&searchIdx=0'}}]

In [17]:
# Open document tree
index = result[0]["search_index"]
retriever.browser.open(
            "https://www.unternehmensregister.de/ureg/registerPortal.html;{}?submitaction=showDkTree&searchIdx={}".format(
                retriever.session_id, index))


<Response [200]>

In [20]:
elements = retriever.browser.page.select("div.dktree-container.level-{} span a".format(2))

In [56]:
def filter_and_follow_link(elements, keyword):
    filtered_elements = list(filter(lambda x: keyword in x.string, elements))
    if filtered_elements:
        retriever.browser.follow_link(filtered_elements[0])
    else:
        print("not found")

def retrieve_documents(path):
    no_iterations = len(path) + 3
    selector = "div.dktree-container.level-{} span a"
    for level in range(2, no_iterations):
        elements = retriever.browser.page.select(selector.format(level))
        print(elements)

        if len(elements) == 0:
            raise Exception("no documents found")
        elif len(elements) == 1:
            retriever.browser.open_relative(elements[0].attrs["href"])

        if level > 2:
            filter_and_follow_link(elements, keyword=path[level-3])

        level += 1
    
    return retriever.browser.page.select(selector.format(level))

path = ["Weitere Urkunden / Unterlagen", "Anmeldung"]
results = retrieve_documents(path)

5
2
[<a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0&amp;searchIdx=0">Dokumente zur Registernummer</a>]
3
[<a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_0&amp;searchIdx=0">Gesellschaftsvertrag / Satzung / Statut</a>, <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_1&amp;searchIdx=0">Liste der Gesellschafter</a>, <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2&amp;searchIdx=0">Weitere Urkunden / Unterlagen</a>, <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_3&amp;searchIdx=0">Anzeige nach Eintragung</a>, <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_4&amp;searchIdx=0">Anzeige nach Eingang</a>]
Weitere Urkunden / Unterlagen
4
[<a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_0_0&amp;searchIdx=0">Gesellschaftsvertrag / Satzung / Statut vom 21.04.2023</a>, <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0

In [57]:
results

[<a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2_0_0&amp;searchIdx=0">Anmeldung vom 11.04.2023</a>,
 <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2_0_1&amp;searchIdx=0">Anmeldung vom 03.04.2023</a>,
 <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2_0_2&amp;searchIdx=0">Anmeldung vom 10.11.2022</a>,
 <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2_0_3&amp;searchIdx=0">Anmeldung vom 20.09.2021</a>,
 <a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2_0_4&amp;searchIdx=0">Anmeldung vom 09.08.2021</a>]

In [64]:
from datetime import datetime
import re

pattern = r"\b(\d{2}\.\d{2}\.\d{4})\b"

# Function to extract the date from the element using regex
def extract_date(element):
    match = re.search(pattern, element.text)
    if match:
        date_string = match.group(1)
        return datetime.strptime(date_string, "%d.%m.%Y")

# Find the oldest document
oldest_document = min(results, key=extract_date)

print(oldest_document)

<a href="?submitaction=showDkTree&amp;action=select&amp;addedId=N__0_0_2_0_4&amp;searchIdx=0">Anmeldung vom 09.08.2021</a>


In [63]:
import re
from datetime import datetime

# Example string
text = "This is an example string with a date 14.07.2023 in the format %d.%m.%Y."

# Regular expression pattern to find a date in the format "%d.%m.%Y"
pattern = r"\b(\d{2}\.\d{2}\.\d{4})\b"

# Find the date using regex
match = re.search(pattern, results[0].text)

if match:
    date_string = match.group(1)
    date = datetime.strptime(date_string, "%d.%m.%Y")
    print(date)
else:
    print("No date found.")


2023-04-11 00:00:00


In [65]:
file_format = [x.attrs["value"] for x in retriever.browser.page.select("input#format_orig")][0]
retriever.browser.select_form("#dkform")
retriever.browser["format"] = file_format
retriever.browser.submit_selected("add2cart")

IndexError: list index out of range